# Sufficient levels computation variants
We have two variants how to compute the $\mathrm{Safe}^\mathit{cap}$ vector (level of energy needed to survive with given capacity).

1. Iterate minInitCons (largest fixpoint) on MDP where reloads with ∞ are removed. \[$n^2$ iterations\]
2. Least fixpoint from minInitCons \[$(n\cdot c_{\mathrm{max}})$ iterations\]


The two variants are implemented currently in [energy_solver.py](energy_solver.py) in:
1. 
```python
class EnergySolver
```

2. 
```python
class EnergyLevels_least
```

The option 2 computes minInitCons as a first step anyway (no reloads are removed at the beginning) as it results in a cleaner code. However, it does not sotre the values.

In [3]:
import sys; sys.path.insert(0, '..')
from cmdp.energy_solver import EnergySolver, EnergyLevels_least

## Examples
* Manhattan
* a tiny MDP used to show incorrectness of the least-fixpoint with at most $|S|$ iterations.

### Manhattan example

In [4]:
from cmdp import ch_parser
man, targets = ch_parser.parse('NYCstreetnetwork.json')
man.structure_change()

### Tiny example

In [5]:
from cmdp import consMDP
m = consMDP.ConsMDP()
m.new_state(True)
m.new_states(2)
m.new_state(True)
m.add_action(0, {0:1}, "", 1)
m.add_action(1, {0:1}, "a", 1000)
m.add_action(1, {2:1}, "b", 1)
m.add_action(2, {1:1}, "b", 1)
m.add_action(3, {3:1}, "r", 1010)
m.add_action(1, {3:1}, "r", 1)
m.add_action(2, {3:1}, "r", 1);

## Variant 1

In [7]:
cap = 40
man.def_EL_class = EnergySolver
%timeit man.get_safe(cap, recompute=True)

5.23 s ± 43.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
m.energy_levels = EnergySolver(m, 1005)
%timeit m.get_safe(1005, recompute=True)

234 µs ± 2.47 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## Variant 2

In [9]:
man.def_EL_class = EnergyLevels_least
%timeit man.get_safe(cap, recompute=True)

1.09 s ± 4.24 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
m.def_EL_class = EnergyLevels_least
%timeit m.get_safe(1005, recompute=True)

15.5 ms ± 37.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Effect on Büchi computation is negligible as it does not use get_safe()

In [11]:
man.def_EL_class = EnergySolver
man.structure_change()
%timeit man.get_Buchi(targets, 100, recompute=True)

4.14 s ± 73.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
m.def_EL_class = EnergyLevels_least
man.structure_change()
%timeit man.get_Buchi(targets, 100, recompute=True)

4.16 s ± 115 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
